In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder

In [2]:
data1 = pd.read_csv("ResaleFlatPricesBasedonApprovalDate19901999.csv")
data2 = pd.read_csv("ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv")
data3 = pd.read_csv("ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv")
data4 = pd.read_csv("ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv")
data5 = pd.read_csv("ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")

In [7]:
data4.drop("remaining_lease",axis=1,inplace=True)
data5.drop("remaining_lease",axis=1,inplace=True)

In [3]:
print(data1.shape)
print(data2.shape)
print(data3.shape)
print(data4.shape)
print(data5.shape)

(287196, 10)
(369651, 10)
(52203, 10)
(37153, 11)
(169071, 11)


In [36]:
data = pd.concat([data1,data2,data3,data4,data5],axis=0,ignore_index=True)

In [37]:
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [38]:
data['remaining_lease'] = data['remaining_lease'].fillna(0)

In [39]:
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,0
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,0
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,0
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,0
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,0


In [40]:
data.shape

(915274, 11)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915274 entries, 0 to 915273
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                915274 non-null  object 
 1   town                 915274 non-null  object 
 2   flat_type            915274 non-null  object 
 3   block                915274 non-null  object 
 4   street_name          915274 non-null  object 
 5   storey_range         915274 non-null  object 
 6   floor_area_sqm       915274 non-null  float64
 7   flat_model           915274 non-null  object 
 8   lease_commence_date  915274 non-null  int64  
 9   resale_price         915274 non-null  float64
 10  remaining_lease      915274 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 76.8+ MB


In [42]:
data.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

In [43]:
data.duplicated().sum()

1892

In [12]:
data.drop_duplicates(inplace=True)

In [24]:
re = list(data['remaining_lease'].unique())
print(re)          

[0.0, 70.0, 65.0, 64.0, 63.0, 62.0, 69.0, 60.0, 61.0, 86.0, 77.0, 80.0, 90.0, 87.0, 66.0, 58.0, 94.0, 71.0, 68.0, 84.0, 73.0, 79.0, 76.0, 72.0, 82.0, 74.0, 67.0, 88.0, 81.0, 89.0, 53.0, 54.0, 55.0, 57.0, 93.0, 83.0, 85.0, 92.0, 91.0, 59.0, 95.0, 52.0, 51.0, 56.0, 75.0, 96.0, 78.0, 50.0, 97.0, 49.0, 48.0, 61.33, 60.58, 62.42, 62.08, 61.5, 58.33, 59.67, 59.5, 62.67, 60.83, 59.25, 61.42, 60.33, 60.25, 63.75, 61.08, 61.83, 58.5, 59.33, 62.92, 60.67, 93.67, 93.58, 60.08, 94.67, 78.33, 60.5, 62.5, 70.67, 63.33, 63.5, 67.58, 61.58, 68.17, 68.25, 67.75, 67.42, 63.58, 66.25, 65.33, 69.42, 59.92, 60.42, 69.17, 69.25, 68.83, 62.83, 64.33, 66.08, 83.08, 87.92, 71.17, 92.33, 54.5, 78.5, 82.92, 75.33, 66.58, 66.5, 75.92, 68.33, 55.75, 68.58, 67.92, 69.08, 69.92, 74.5, 74.33, 69.5, 72.25, 67.17, 66.42, 69.33, 66.92, 66.83, 69.67, 66.75, 67.83, 80.08, 67.5, 86.67, 71.5, 71.25, 67.33, 86.92, 86.83, 79.42, 65.83, 67.25, 79.92, 53.5, 57.17, 52.08, 58.25, 51.5, 58.67, 56.17, 53.67, 64.67, 55.5, 95.58, 55.

In [27]:
data['remaining_lease'].dtypes

dtype('float64')

In [44]:
def convert_year(x):
    year = 0
    if type(x) == int:
        return x
    if type(x) == str:
        s = x.split()        
        if len(s) == 4:
            year += int(s[0])
            month = int(s[2])
            year += round(month/12,2)
        if len(s) == 2:
            year += int(s[0])       
    return year
convert_year(86) 

86

In [45]:
data['remaining_lease'] = data['remaining_lease'].apply(convert_year)

In [53]:
data['resale_year'] = pd.to_datetime(data['month'], format="%Y-%M").dt.year
data['resale_month'] = pd.to_datetime(data['month'], format="%Y-%M").dt.month

In [54]:
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_low,storey_up,resale_year,resale_month
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,0.0,10,12,1990,1
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,0.0,4,6,1990,1
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,0.0,10,12,1990,1
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,0.0,7,9,1990,1
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,0.0,4,6,1990,1


In [48]:
data['storey_range'].unique()

array(['10 TO 12', '04 TO 06', '07 TO 09', '01 TO 03', '13 TO 15',
       '19 TO 21', '16 TO 18', '25 TO 27', '22 TO 24', '28 TO 30',
       '31 TO 33', '40 TO 42', '37 TO 39', '34 TO 36', '06 TO 10',
       '01 TO 05', '11 TO 15', '16 TO 20', '21 TO 25', '26 TO 30',
       '36 TO 40', '31 TO 35', '46 TO 48', '43 TO 45', '49 TO 51'],
      dtype=object)

In [50]:
def low(x):
    if type(x) == str:
        s = x.split('TO')
        return int(s[0])
def up(x):
    if type(x) == str:
        s = x.split('TO')
        return int(s[1])
low('10 TO 12')
up('10 TO 12')

12

In [51]:
data['storey_low'] = data['storey_range'].apply(low)
data['storey_up'] = data['storey_range'].apply(up)

In [52]:
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_low,storey_up
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,0.0,10,12
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,0.0,4,6
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,0.0,10,12
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,0.0,7,9
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,0.0,4,6


In [55]:
data.drop(['month','storey_range'],axis=1,inplace=True)

In [57]:
data.to_csv("resale.csv")

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915274 entries, 0 to 915273
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   town                 915274 non-null  object 
 1   flat_type            915274 non-null  object 
 2   block                915274 non-null  object 
 3   street_name          915274 non-null  object 
 4   floor_area_sqm       915274 non-null  float64
 5   flat_model           915274 non-null  object 
 6   lease_commence_date  915274 non-null  int64  
 7   resale_price         915274 non-null  float64
 8   remaining_lease      915274 non-null  float64
 9   storey_low           915274 non-null  int64  
 10  storey_up            915274 non-null  int64  
 11  resale_year          915274 non-null  int32  
 12  resale_month         915274 non-null  int32  
dtypes: float64(3), int32(2), int64(3), object(5)
memory usage: 83.8+ MB


In [58]:
for col in data.columns:
    if data[col].dtypes == "object":
        data[col] = pd.Categorical(data[col]).codes

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915274 entries, 0 to 915273
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   town                 915274 non-null  int8   
 1   flat_type            915274 non-null  int8   
 2   block                915274 non-null  int16  
 3   street_name          915274 non-null  int16  
 4   floor_area_sqm       915274 non-null  float64
 5   flat_model           915274 non-null  int8   
 6   lease_commence_date  915274 non-null  int64  
 7   resale_price         915274 non-null  float64
 8   remaining_lease      915274 non-null  float64
 9   storey_low           915274 non-null  int64  
 10  storey_up            915274 non-null  int64  
 11  resale_year          915274 non-null  int32  
 12  resale_month         915274 non-null  int32  
dtypes: float64(3), int16(2), int32(2), int64(3), int8(3)
memory usage: 55.0 MB


In [60]:
data.head()

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_low,storey_up,resale_year,resale_month
0,0,0,829,13,31.0,7,1977,9000.0,0.0,10,12,1990,1
1,0,0,829,13,31.0,7,1977,6000.0,0.0,4,6,1990,1
2,0,0,829,13,31.0,7,1977,8000.0,0.0,10,12,1990,1
3,0,0,829,13,31.0,7,1977,6000.0,0.0,7,9,1990,1
4,0,2,447,13,73.0,20,1976,47200.0,0.0,4,6,1990,1


In [61]:
x = data.drop('resale_price',axis=1)
y = data.pop('resale_price')

In [62]:
x.head()

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,storey_low,storey_up,resale_year,resale_month
0,0,0,829,13,31.0,7,1977,0.0,10,12,1990,1
1,0,0,829,13,31.0,7,1977,0.0,4,6,1990,1
2,0,0,829,13,31.0,7,1977,0.0,10,12,1990,1
3,0,0,829,13,31.0,7,1977,0.0,7,9,1990,1
4,0,2,447,13,73.0,20,1976,0.0,4,6,1990,1


In [63]:
y.head()

0     9000.0
1     6000.0
2     8000.0
3     6000.0
4    47200.0
Name: resale_price, dtype: float64

In [64]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=1)

In [65]:
dtr = DecisionTreeRegressor()
model = dtr.fit(xtrain,ytrain)

In [66]:
pred = model.predict(xtest)
print(r2_score(ytest,pred))

0.9649075208668497


In [67]:
pred1 = model.predict(xtrain)
print(r2_score(ytrain,pred1))

0.9983148446940033


In [69]:
from joblib import dump

In [70]:
dump(model,"resalemodel.joblib")

['resalemodel.joblib']

In [16]:
data = pd.read_csv('resale.csv')
data.head()

,Unnamed: 0,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_low,storey_up,resale_year,resale_month
0,0,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,9000.0,0.0,10,12,1990,1
1,1,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,6000.0,0.0,4,6,1990,1
2,2,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,8000.0,0.0,10,12,1990,1
3,3,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,6000.0,0.0,7,9,1990,1
4,4,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,73.0,NEW GENERATION,1976,47200.0,0.0,4,6,1990,1


In [18]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [19]:
data.head()

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_low,storey_up,resale_year,resale_month
0,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,9000.0,0.0,10,12,1990,1
1,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,6000.0,0.0,4,6,1990,1
2,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,8000.0,0.0,10,12,1990,1
3,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,6000.0,0.0,7,9,1990,1
4,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,73.0,NEW GENERATION,1976,47200.0,0.0,4,6,1990,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915274 entries, 0 to 915273
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   town                 915274 non-null  object 
 1   flat_type            915274 non-null  object 
 2   block                915274 non-null  object 
 3   street_name          915274 non-null  object 
 4   floor_area_sqm       915274 non-null  float64
 5   flat_model           915274 non-null  object 
 6   lease_commence_date  915274 non-null  int64  
 7   resale_price         915274 non-null  float64
 8   remaining_lease      915274 non-null  float64
 9   storey_low           915274 non-null  int64  
 10  storey_up            915274 non-null  int64  
 11  resale_year          915274 non-null  int64  
 12  resale_month         915274 non-null  int64  
dtypes: float64(3), int64(5), object(5)
memory usage: 90.8+ MB


In [14]:
print(data['flat_model'].nunique())
town = list(data['flat_model'].unique())
town.sort()
print(town)

34
['2-ROOM', '2-room', '3Gen', 'APARTMENT', 'Adjoined flat', 'Apartment', 'DBSS', 'IMPROVED', 'IMPROVED-MAISONETTE', 'Improved', 'Improved-Maisonette', 'MAISONETTE', 'MODEL A', 'MODEL A-MAISONETTE', 'MULTI GENERATION', 'Maisonette', 'Model A', 'Model A-Maisonette', 'Model A2', 'Multi Generation', 'NEW GENERATION', 'New Generation', 'PREMIUM APARTMENT', 'Premium Apartment', 'Premium Apartment Loft', 'Premium Maisonette', 'SIMPLIFIED', 'STANDARD', 'Simplified', 'Standard', 'TERRACE', 'Terrace', 'Type S1', 'Type S2']


In [25]:
data['floor_area_sqm'].max()

307.0